In [1]:
from torchvision.models import *
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Hutton-Rock-Dataset'

In [2]:
def load_data():
    data = []
    labels = {}
    labels_r = {}
    idx = 0
    for folder in os.listdir('./data/'):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
    print(labels)
    print(idx)
    for folder in tqdm(os.listdir('./data/')):
        print(folder)
        for file in tqdm(os.listdir(f'./data/{folder}')):
            try:
                img = cv2.imread(f'./data/{folder}/{file}')
                img = cv2.resize(img,(112,112))
                img = img / 255.0
                data.append([img,np.eye(labels[folder],idx)[labels[folder]-1]])
            except:
                pass
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False,random_state=2008)
    X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
    X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
    y_train = torch.from_numpy(np.array(y_train)).to(device).float()
    y_test = torch.from_numpy(np.array(y_test)).to(device).float()
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [3]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

{'basalt': 1, 'granite': 2, 'andesite': 3}
3


  0%|                                                     | 0/3 [00:00<?, ?it/s]

basalt



 33%|███████████████                              | 1/3 [00:00<00:00,  3.47it/s]

granite



 67%|██████████████████████████████               | 2/3 [00:00<00:00,  2.36it/s]

andesite



100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.56it/s]


In [4]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [5]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        y_batch = int(torch.argmax(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [ ]:
model = resnet34().to(device)

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 11.76 GiB total capacity; 8.02 GiB already allocated; 3.50 MiB free; 8.21 GiB reserved in total by PyTorch)

In [ ]:
model.fc

In [ ]:
model.fc = Linear(2048,3)

In [ ]:
model.fc

In [ ]:
criterion = MSELoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [30]:
batch_size = 32
epochs = 100

In [31]:
torch.cuda.empty_cache()
wandb.init(project=PROJECT_NAME,name='baseline')
torch.cuda.empty_cache()
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for i in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[i:i+batch_size].to(device)
        y_batch = y_train[i:i+batch_size].to(device)
        model.to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
torch.cuda.empty_cache()
wandb.finish()
torch.cuda.empty_cache()

  0%|                                                   | 0/100 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 60.00 MiB (GPU 0; 11.76 GiB total capacity; 7.95 GiB already allocated; 43.31 MiB free; 8.18 GiB reserved in total by PyTorch)